In [1]:
import os
import subprocess
import sys
import warnings

import alphashape
import gmsh
import matplotlib.pyplot as plt
import meshio
import numpy as np
import pandas as pd

import commons, geometry, grapher, utils

In [2]:
img_id = 11
scale = (800.0 / 470.0) * 1e-6
scale_x = scale
scale_y = scale
scale_z = scale
Lsep = 25 * scale
Lcat = 50 * scale
LX = 470 * scale
LY = 470 * scale
LZ = int((Lsep + Lcat)/scale)
Rp = 6 * scale
eps_se = 0.5
eps_am = 1 - eps_se
markers = commons.Markers()
CELL_TYPES = commons.CellTypes()
resolution = 1
start_zpos = 50
scale_factor = [scale, scale, scale]
dimensions = f'470-470-{LZ}'
print(dimensions)
name_of_study = 'reaction_distribution'
outdir = f"output/{name_of_study}/{dimensions}/{img_id}/{eps_am}/{resolution}"
utils.make_dir_if_missing(outdir)
msh_path = os.path.join(outdir, 'mesh.msh')

470-470-75


## Build Laminate Geometry of Cylindrical SE Strands

In [3]:
df = scale * 470 * pd.read_csv('data/laminate.csv')

In [4]:
z0_points = [
        (0, 0, 0),
        (LX, 0, 0),
        (LX, LY, 0),
        (0, LY, 0),
    ]
points0 = []
gmsh.initialize()
gmsh.model.add('laminate')
gmsh.option.setNumber('General.Verbosity', 1)
left_active = []
side_loops = []
lines_left = []
right = []
left = []
insulated = []
insulated_se = []
insulated_am = []
interface = []

for i in range(4):
    idx = gmsh.model.occ.addPoint(*z0_points[i])
    points0.append(idx)
lines_left.append(
    gmsh.model.occ.addLine(points0[0], points0[1])
)
lines_left.append(
    gmsh.model.occ.addLine(points0[1], points0[2])
)
lines_left.append(
    gmsh.model.occ.addLine(points0[2], points0[3])
)
lines_left.append(
    gmsh.model.occ.addLine(points0[3], points0[0])
)
left_loop = gmsh.model.occ.addCurveLoop(lines_left)
gmsh.model.occ.synchronize()
img = np.asarray(plt.imread(f'data/current_constriction/test{str(int(img_id))}.tif')[:, :, 0], dtype=np.uint8)
image = img.copy()
image[0, :] = 0
image[-1, :] = 0
image[:, 0] = 0
image[:, -1] = 0
boundary_pieces, count, points, points_view = geometry.get_phase_boundary_pieces(image)
for hull in boundary_pieces:
    hull_arr = np.asarray(hull)
    hull_points = []
    for pp in hull[:-1]:
        idx = gmsh.model.occ.addPoint(int(pp[0]) * scale_x, int(pp[1]) * scale_y, 0)
        hull_points.append(
            idx
        )
    gmsh.model.occ.synchronize()
    hull_lines = []
    for i in range(-1, len(hull_points) - 1):
        idx = gmsh.model.occ.addLine(hull_points[i], hull_points[i + 1])
        hull_lines.append(
            idx
        )

    gmsh.model.occ.synchronize()
    idx = gmsh.model.occ.addCurveLoop(hull_lines)
    side_loops.append(idx)
    idx2 = gmsh.model.occ.addPlaneSurface((idx, ))
    left_active.append(idx2)
    gmsh.model.occ.synchronize()

left_inactive = gmsh.model.occ.addPlaneSurface((left_loop, *side_loops))
gmsh.model.occ.synchronize()
insulated_se.append(left_inactive)

box_se = gmsh.model.occ.addBox(0, 0, 0, LX, LY, Lsep + Lcat - Rp)
gmsh.model.occ.synchronize()
box_am = gmsh.model.occ.addBox(0, 0, Lsep + Lcat - Rp, LX, LY, Rp)
gmsh.model.occ.synchronize()
cylinders = []
spheres = []

for idx in range(df.shape[0]):
    x, y, _ = df.loc[idx, :]
    if (x + Rp) >= LX or (y + Rp) >= LY:
        continue
    if (x - Rp) <= 0 or (y - Rp) <= 0:
        continue
    cyl = gmsh.model.occ.addCylinder(x, y, Lsep, 0, 0, Lcat - Rp, Rp)
    gmsh.model.occ.synchronize()
    cylinders.append(cyl)

union = gmsh.model.occ.fuse([(3, box_am)], [(3, c) for c in cylinders], removeTool=False)
gmsh.model.occ.synchronize()
se_phase = gmsh.model.occ.cut([(3, box_se)], [(3, c) for c in cylinders], removeTool=False)
gmsh.model.occ.synchronize()
vols = gmsh.model.occ.getEntities(3)
se_volumes = []
am_volumes = []
for (_, vol) in vols:
    com = gmsh.model.occ.getCenterOfMass(3, vol)
    z = com[2] / scale
    if np.isclose(z, 0.5 * (Lsep + Lcat - Rp) / scale, atol=1):
        se_volumes.append(vol)
    else:
        am_volumes.append(vol)
se_vol = gmsh.model.addPhysicalGroup(3, se_volumes, markers.electrolyte)
gmsh.model.occ.synchronize()
gmsh.model.setPhysicalName(3, se_vol, "electrolyte")
gmsh.model.occ.synchronize()
am_vol = gmsh.model.addPhysicalGroup(3, am_volumes, markers.positive_am)
gmsh.model.occ.synchronize()
gmsh.model.setPhysicalName(3, am_vol, "positive_am")
gmsh.model.occ.synchronize()

for surf in gmsh.model.occ.getEntities(2):
    com = gmsh.model.occ.getCenterOfMass(surf[0], surf[1])
    x = com[0] / scale
    y = com[1] / scale
    z = com[2] / scale
    if np.isclose(z, 0, atol=1):
        left.append(surf[1])
    elif np.isclose(z, (Lsep + Lcat)/scale, atol=1):
        right.append(surf[1])
    elif np.isclose(z, 0.5 * (Lsep + Lcat - Rp) / scale, atol=1):
        if np.isclose(x, LX/scale, atol=1) or np.isclose(y, LY/scale, atol=1) or np.isclose(x, 0, atol=1) or np.isclose(y, 0, atol=1):
            insulated_se.append(surf[1])
        else:
            interface.append(surf[1])
    elif np.isclose(z, (Lsep + Lcat - 0.5 * Rp) / scale, atol=1):
        if np.isclose(x, LX/scale, atol=1) or np.isclose(y, LY/scale, atol=1) or np.isclose(x, 0, atol=1) or np.isclose(y, 0, atol=1):
            insulated_am.append(surf[1])
        else:
            interface.append(surf[1])
    else:
        interface.append(surf[1])

left = gmsh.model.addPhysicalGroup(2, left_active, markers.left)
gmsh.model.setPhysicalName(2, left, "left")
right = gmsh.model.addPhysicalGroup(2, right, markers.right)
gmsh.model.setPhysicalName(2, right, "right")
insulated_se = gmsh.model.addPhysicalGroup(2, insulated_se, markers.insulated_electrolyte)
gmsh.model.setPhysicalName(2, insulated_se, "insulated_electrolyte")
insulated_am = gmsh.model.addPhysicalGroup(2, insulated_am, markers.insulated_positive_am)
gmsh.model.setPhysicalName(2, insulated_am, "insulated_am")
insulated = gmsh.model.addPhysicalGroup(2, insulated, markers.insulated)
gmsh.model.setPhysicalName(2, insulated, "insulated")
electrolyte_v_positive_am = gmsh.model.addPhysicalGroup(2, interface, markers.electrolyte_v_positive_am)
gmsh.model.setPhysicalName(2, electrolyte_v_positive_am, "electrolyte_positive_am_interface")
gmsh.model.occ.synchronize()
gmsh.model.mesh.generate(3)
gmsh.write(msh_path)
gmsh.finalize()

4 QH6013 qhull input error: input is less than 3-dimensional since all points have the same x coordinate   70

While executing:  | qhull d Q12 Qbb Qt Qc Qz
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 1047693977  delaunay  Q12-allow-wide  Qbbound-last  Qtriangulate
  Qcoplanar-keep  Qz-infinity-point  _pre-merge  _zero-centrum  Qinterior-keep
  Pgood  _max-width  3  Error-roundoff 5.8e-13  _one-merge 4.1e-12
  Visible-distance 1.2e-12  U-max-coplanar 1.2e-12  Width-outside 2.3e-12
  _wide-facet 7e-12  _maxoutside 4.6e-12

